In [ ]:
%matplotlib ipympl

In [ ]:
import scipp as sc

from ess.beer import BeerMcStasWorkflow
from ess.beer.data import mcstas_silicon_medium_resolution, mcstas_duplex_medium_resolution
from ess.reduce.nexus.types import Filename, SampleRun
from ess.reduce.time_of_flight.types import DetectorTofData

In [ ]:
wf = BeerMcStasWorkflow()

In [ ]:
wf[Filename[SampleRun]] = mcstas_duplex_medium_resolution()
da = wf.compute(DetectorTofData[SampleRun])
da.bins.coords['d'] = (sc.constants.h / sc.constants.m_n * da.bins.coords['tof'] / da.bins.coords['sin_theta_L'] / 2).to(unit='angstrom')
da.bins.concat().hist(d=2000).plot()

In [ ]:
wf[Filename[SampleRun]] = mcstas_silicon_medium_resolution()
da = wf.compute(DetectorTofData[SampleRun])
da.bins.coords['d'] = (sc.constants.h / sc.constants.m_n * da.bins.coords['tof'] / da.bins.coords['sin_theta_L'] / 2).to(unit='angstrom')
da.bins.concat().hist(d=2000).plot()